# Installing Packages

In [2]:
import os
import gc
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, levene, ranksums
from sklearn.linear_model import LinearRegression
import numpy as np
import pyBigWig
import math
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
from statsmodels.multivariate.manova import MANOVA
from scipy import stats
import statsmodels.api as sm
from matplotlib import gridspec
from matplotlib.patches import Patch
import matplotlib.colors as mcolors
import glob
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import importlib
import config 
importlib.reload(config)
from config import BIN_SIZE 

# Loading Samples (30 Holdout-Dataset)

In [3]:
cancer_samples = [
    # bile duct cancer
    "EE87786", "EE87787", "EE87788",

    # colorectal 
    "EE85723", "EE85724", "EE85726",

    #duodenal
    "EE87892",

    #eso
    "EE86253", "EE86266",

    #gastric
    "EE87893", "EE87894", "EE87895",

    #pancreatic
    "EE86244", "EE86247", "EE86248",

]
control_samples = [
    "EE85756",
    "EE85757",
    "EE85770",
    "EE85775",
    "EE85784",
    "EE85787",
"EE85791",
"EE85795",
"EE85816",
"EE85830",
"EE85842",
"EE85854",
"EE85860",
"EE85861",
"EE85878",

]

BASE_DIR = "/labmed/workspace/lotta/finaletoolkit/carsten/outputs_holdout"

def find_sample_folder(sample, base_dir=BASE_DIR):
    for root, dirs, files in os.walk(base_dir):
        for f in files:
            if f.startswith(sample) and f.endswith(".adjust_wps.bw"):
                return root
    return None

def get_bigwig_path(sample):
    folder = find_sample_folder(sample)
    if folder is None:
        raise FileNotFoundError(f"Sample {sample} not found in {BASE_DIR}")
    return os.path.join(folder, f"{sample}.adjust_wps.bw")

def bigwig_summary(bigwig_path, chrom, start, end, n_bins=1):
    bw = pyBigWig.open(bigwig_path)
    bin_size = (end - start) // n_bins
    results = []
    
    for i in range(n_bins):
        b_start = start + i * bin_size
        b_end = start + (i+1) * bin_size if i < n_bins - 1 else end
        
        vals = bw.values(chrom, b_start, b_end)
        vals = [v for v in vals if v is not None and not math.isnan(v)]
        
        results.append(sum(vals)/len(vals) if vals else 0)

    bw.close()
    return results

all_samples = cancer_samples + control_samples
print(f"Configuration loaded for {len(all_samples)} samples:")
print(all_samples)

Configuration loaded for 30 samples:
['EE87786', 'EE87787', 'EE87788', 'EE85723', 'EE85724', 'EE85726', 'EE87892', 'EE86253', 'EE86266', 'EE87893', 'EE87894', 'EE87895', 'EE86244', 'EE86247', 'EE86248', 'EE85756', 'EE85757', 'EE85770', 'EE85775', 'EE85784', 'EE85787', 'EE85791', 'EE85795', 'EE85816', 'EE85830', 'EE85842', 'EE85854', 'EE85860', 'EE85861', 'EE85878']


# Cancer Typ aus dem Pfad extrahieren

In [4]:
def get_cancer_type(sample):
    folder = find_sample_folder(sample)  
    if folder is None:
        return "Unknown"
    return os.path.basename(folder) 

# Creating and Loading of Bedgraph Files 

# Bin-Wide-Analysis, Binning the genome, Bin Size in Config File 


In [7]:
bedgraph_dir = os.path.expanduser('/labmed/workspace/lotta/finaletoolkit/carsten/outputs_holdout/')
from config import BIN_SIZE
print(BIN_SIZE)

binned_output_path = f"/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/dataframes_holdout/binned_combined_df_{BIN_SIZE}.parquet"

all_binned_dfs = []

if os.path.exists(binned_output_path):
    print(f"Loading existing binned dataframe from {binned_output_path}...")
    binned_combined_df = pd.read_parquet(binned_output_path)
else:
    print(f"Creating new binned dataframe with bin size {BIN_SIZE}...")
    
    def find_bedgraphs(sample_id):
        # pattern ist der gesuchte Dateipfad
        pattern = os.path.join(bedgraph_dir, "**", f"{sample_id}.adjust_wps.bedgraph")

        # matches sind alle gefundenen Dateien, die dem Muster entsprechen
        matches = glob.glob(pattern, recursive=True)
        # Gibt die erste gefundene Datei zurück 
        return matches[0] if matches else None

    for sample_id in all_samples:
        file_path = find_bedgraphs(sample_id)
        if file_path:
            try:
                df = pd.read_csv(file_path, sep="\t", header=None, names=["chrom", "start", "end", "wps_value"])
                df['sample'] = sample_id
                group = get_cancer_type(sample_id)
                df['group'] = group
                
                # IMMEDIATE BINNING TO SAVE MEMORY
                df['bin'] = df['start'] // BIN_SIZE
                # Calculate mean per bin for this sample immediately
                df_binned = df.groupby(['sample', 'group', 'chrom', 'bin'])['wps_value'].mean().reset_index()
                
                all_binned_dfs.append(df_binned)
                print(f"Loaded and binned {sample_id}. Rows: {len(df)} -> {len(df_binned)}")
                
                del df
                gc.collect()
            except Exception as e:
                print(f"Error processing {sample_id}: {e}")
        else:
            print(f"Bedgraph file for sample {sample_id} not found.")

    if all_binned_dfs:
        binned_combined_df = pd.concat(all_binned_dfs, ignore_index=True)
        print(f"Data successfully loaded and binned. Total rows: {len(binned_combined_df)}")
        
        # Apply median imputation for (chrom, bin) groups
               # Check for NaN values before imputation
        nan_count = binned_combined_df['wps_value'].isna().sum()
        print(f"Number of NaN values before imputation: {nan_count}")

        if nan_count > 0:
            print("Applying median imputation...")
            binned_combined_df['wps_value'] = binned_combined_df.groupby(['chrom', 'bin'])['wps_value'].transform(lambda x: x.fillna(x.median()))
        else:
            print("No NaN values found. Skipping imputation.")
        binned_combined_df.to_parquet(binned_output_path)
        print(f"Saved binned dataframe to {binned_output_path}")
    else:
        print("No data found!")


50000
Creating new binned dataframe with bin size 50000...
Loaded and binned EE87786. Rows: 26110000 -> 4285
Loaded and binned EE87787. Rows: 26110000 -> 4285
Loaded and binned EE87788. Rows: 26110000 -> 4285
Loaded and binned EE85723. Rows: 26110000 -> 4285
Loaded and binned EE85724. Rows: 26110000 -> 4285
Loaded and binned EE85726. Rows: 26110000 -> 4285
Loaded and binned EE87892. Rows: 26110000 -> 4285
Loaded and binned EE86253. Rows: 26110000 -> 4285
Loaded and binned EE86266. Rows: 26110000 -> 4285
Loaded and binned EE87893. Rows: 26110000 -> 4285
Loaded and binned EE87894. Rows: 26110000 -> 4285
Loaded and binned EE87895. Rows: 26110000 -> 4285
Loaded and binned EE86244. Rows: 26110000 -> 4285
Loaded and binned EE86247. Rows: 26110000 -> 4285
Loaded and binned EE86248. Rows: 26110000 -> 4285
Loaded and binned EE85756. Rows: 26110000 -> 4285
Loaded and binned EE85757. Rows: 26110000 -> 4285
Loaded and binned EE85770. Rows: 26110000 -> 4285
Loaded and binned EE85775. Rows: 26110000

# Feature Matrix for LR rows=sample and columns=bins+groups 


In [8]:
binned_combined_df = pd.read_parquet(f"/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/dataframes_holdout/binned_combined_df_{BIN_SIZE}.parquet")
if os.path.exists(f"/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/dataframes_holdout/final_feature_matrix_{BIN_SIZE}.parquet"):
    print("Loading existing final feature matrix...")
    final_feature_matrix = pd.read_parquet(f"/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/dataframes_holdout/final_feature_matrix_{BIN_SIZE}.parquet")
else:
    binned_combined_df['feature_name'] = binned_combined_df['chrom'] + '_bin_' + binned_combined_df['bin'].astype(str)
    feature_matrix = binned_combined_df.pivot(index='sample', columns='feature_name', values='wps_value')
    group_info = binned_combined_df[['sample', 'group']].drop_duplicates().set_index('sample')
    final_feature_matrix = feature_matrix.join(group_info)
    final_feature_matrix = final_feature_matrix.fillna(0)
    final_feature_matrix.to_parquet(f"/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/dataframes_holdout/final_feature_matrix_{BIN_SIZE}.parquet", index=True)
    print(final_feature_matrix.head())

         chr10_bin_1008  chr10_bin_101  chr10_bin_1055  chr10_bin_117  \
sample                                                                  
EE85723       -0.670000      -0.656800       -0.336600      -0.489800   
EE85724       -0.457054      -0.587671       -0.205400      -0.570200   
EE85726       -0.726994      -0.792883       -0.532600      -0.541400   
EE85756       -0.349696      -0.356106       -0.297800      -0.619643   
EE85757       -0.307600      -0.029305        0.148367      -0.223800   

         chr10_bin_1170  chr10_bin_118  chr10_bin_1186  chr10_bin_1187  \
sample                                                                   
EE85723       -0.286178      -0.941986       -0.500176       -0.886115   
EE85724       -1.572851      -0.954818       -0.131265       -1.073255   
EE85726       -1.269734      -1.000476       -1.593978       -0.221978   
EE85756       -0.402643       0.171563       -0.220282       -0.200454   
EE85757        0.252263      -0.436200      

# Fragment Interval Analysis: Loading Files


In [10]:
frag_interval_dir = os.path.expanduser('/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/frag_intervals')
frag_intervals_results = []
for sample in all_samples:
    interval_path = os.path.join(frag_interval_dir, '**', f"{sample}.frag_length_intervals.bed")
    files = glob.glob(interval_path, recursive=True)
    if not files:
        print(f"Fragment length Interval file for sample {sample} not found.")
        continue

    df = pd.read_csv(
    files[0],
    sep="\t",
    header=None,
    names=["chrom", "start", "stop", "name", "mean", "median", "stdev", "min", "max"]
    )
    df = df.iloc[1:].reset_index(drop=True)
    group = get_cancer_type(sample)
    df['sample'] = sample
    df['group'] = group
    df["start"] = df["start"].astype(int)
    df["stop"] = df["stop"].astype(int)

    num_cols = ["mean", "median", "stdev", "min", "max"]
    df[num_cols] = df[num_cols].astype(float)
    df['bin'] = df['start'] // BIN_SIZE
    frag_intervals_results.append(df)

frag_intervals_df = pd.concat(frag_intervals_results, ignore_index=True)

In [11]:
print(frag_intervals_df.head())

  chrom    start     stop name        mean  median      stdev    min    max  \
0  chr1   920000   925000    .  166.857143   166.5  19.576433  101.0  224.0   
1  chr1  1070000  1075000    .  167.150000   168.5  15.642970  122.0  202.0   
2  chr1  1165000  1170000    .  165.625000   163.5  14.876350  131.0  209.0   
3  chr1  1170000  1175000    .  168.523077   166.0  14.124836  141.0  217.0   
4  chr1  1175000  1180000    .  167.645161   167.5  15.520119  124.0  204.0   

    sample     group  bin  
0  EE87786  bileduct   18  
1  EE87786  bileduct   21  
2  EE87786  bileduct   23  
3  EE87786  bileduct   23  
4  EE87786  bileduct   23  


# Binning Fragment Interval Files


In [12]:
binned_df = (
    frag_intervals_df.groupby(['sample', 'group', 'chrom', 'bin'])
      .agg({
          "mean": "mean",
          "median": "mean",
          "stdev": "mean",
          "min": "mean",
          "max": "mean"
      })
      .reset_index()
)

print(binned_df.head())
print(binned_df.shape)


    sample       group chrom  bin        mean      median      stdev  \
0  EE85723  colorectal  chr1   18  109.333333   64.000000  32.887012   
1  EE85723  colorectal  chr1   21  111.000000  111.000000   0.000000   
2  EE85723  colorectal  chr1   23   96.250000  100.333333  21.056202   
3  EE85723  colorectal  chr1   25  120.750000  120.750000   6.250000   
4  EE85723  colorectal  chr1   26   91.666667   89.333333   6.374871   

          min         max  
0   64.000000  141.000000  
1  111.000000  111.000000  
2   57.333333  122.333333  
3  114.500000  127.000000  
4   85.333333  102.666667  
(128550, 9)


In [21]:
#(binned_df[["mean","median","stdev","min","max"]] == -1).sum()
(binned_df["mean"] == -1).mean()



np.float64(0.00690003889537145)

In [13]:
print(binned_combined_df.head())

    sample     group chrom  bin  wps_value feature_name
0  EE87786  bileduct  chr1   18  -0.165352  chr1_bin_18
1  EE87786  bileduct  chr1   21  -0.520809  chr1_bin_21
2  EE87786  bileduct  chr1   23  -0.041768  chr1_bin_23
3  EE87786  bileduct  chr1   25  -0.163598  chr1_bin_25
4  EE87786  bileduct  chr1   26  -0.238005  chr1_bin_26


In [14]:
merged_df = pd.merge(
    binned_df,
    binned_combined_df[['sample', 'chrom', 'bin', 'wps_value']],
    how='left',
    on=['sample', 'chrom', 'bin']
)

print(merged_df.head())
merged_df.to_csv(f"/labmed/workspace/lotta/finaletoolkit/ba_analysis_scripts/holdout_preprocessing/dataframes_holdout/final_feature_matrix_{BIN_SIZE}.tsv", sep="\t", index=False)


    sample       group chrom  bin        mean      median      stdev  \
0  EE85723  colorectal  chr1   18  109.333333   64.000000  32.887012   
1  EE85723  colorectal  chr1   21  111.000000  111.000000   0.000000   
2  EE85723  colorectal  chr1   23   96.250000  100.333333  21.056202   
3  EE85723  colorectal  chr1   25  120.750000  120.750000   6.250000   
4  EE85723  colorectal  chr1   26   91.666667   89.333333   6.374871   

          min         max  wps_value  
0   64.000000  141.000000  -0.091200  
1  111.000000  111.000000   0.000000  
2   57.333333  122.333333  -0.128733  
3  114.500000  127.000000  -0.046586  
4   85.333333  102.666667  -0.072200  
